In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import random
import time
from scipy.optimize import *


# !pip install pyMCFSimplex
from pyMCFSimplex import *

#### 1. Estrazione dei dati dai file .dmx e .qfc utilizzati per la generazione delle istanze del problema
La prima funzione *genera_Q(alpha, u, q, dimensione, p)* serve per generare i costi quadratici 
che si aggiungono ai costi lineari dell'istanza di MCF;
la stategia di generazione prevede di generare randomicamente (uniformemente) costi Qij "piccoli" o "grandi" rispetto ai costi lineari, 
come specificato dal parametro *alpha*. 

Permette inoltre di definire - tramite il parametro *p* - la proporzione di archi che hanno costo quadratico Qij nullo, 
abilitando la definizione e l'analisi anche di matrici semi-definite positive. 

La seconda funzione *leggi_file_dimacs(nome_file)* serve per estrarre le seguenti quantità: 
 - u, b, q
 - numero nodi
 - numero archi 

dal relativo file *.dmx*, generato [qui](https://commalab.di.unipi.it/datasets/mcf/). 

In [ ]:
def genera_Q(alpha, u, q, dimensione, p):
    
    Q_diag = []

    for i in range(dimensione):
        Q_i = abs(random.uniform((-q[i] / u[i] * alpha), (q[i] / u[i] * alpha))) # generazione randomica 
        Q_diag.append(Q_i)

    Q = np.zeros((dimensione, dimensione))
    np.fill_diagonal(Q, Q_diag) # creazione matrice 

    num_entrate_zero = int(p * dimensione) 
    indici_zeri = np.random.choice(dimensione, num_entrate_zero, replace=False) # trova p*dimensione indici dei costi quadratici da mettere a zero 

    for idx in indici_zeri:
        Q[idx, idx] = 0
        # Q_diag[idx] = 0

    return Q #, Q_diag


def leggi_file_dimacs(nome_file):
    numero_nodi = 0
    numero_archi = 0
    u = []
    b = []
    q = []
    from_= []
    to_= []
    edges = []

    with open(nome_file, 'r') as file:
        for line in file:
            parts = line.split()
            if len(parts) > 0:
                if parts[0] == 'p':  # parts[0] è il primo carattere di ogni riga - nel formato standard DIMACS può essere c, p, n, a
                    # legge il numero di nodi e archi dal problema
                    numero_nodi = int(parts[2])
                    numero_archi = int(parts[3])
                    # inizializza il vettore di supply con zeri
                    b = [0] * numero_nodi
                elif parts[0] == 'n':
                    # legge i valori di supply per i nodi
                    nodo_id = int(parts[1])
                    supply = int(parts[2])
                    # assegna il valore di supply al nodo corrispondente
                    b[nodo_id - 1] = supply
                elif parts[0] == 'a':
                    # leggi l'arco e il suo cotso
                    from_node = int(parts[1])
                    to_node = int(parts[2])
                    max_capacity = int(parts[4])
                    costo = int(parts[5])  # leggiamo costo corretto
                    from_.append(from_node)
                    to_.append(to_node)
                    u.append(max_capacity)
                    q.append(costo)
                    edges.append((from_node , to_node ))

    return numero_nodi, numero_archi, u, b, q, edges,from_, to_

#### 2. Algoritmo FW

Step: 

0. Inizializzazione di x<sub>0</sub> con generazione casuale soggetta al vincolo 0 &le; x<sub>0</sub> &le; u
1. Calcolo del gradiente, risoluzione del sottoproblema lineare usando il solver pyMCFSimplex (x̄), determinazione della direzione
2. Determinazione dello step size &alpha;
3. Aggiornamento della posizione e check terminazione sul prodotto scalare (<grad, d>)

In [ ]:
# Questa funzione crea un nuovo file .dmx in cui i valori delle righe che iniziano con 'a' (definizione archi)
# vengono sostituiti con i valori presenti nel vettore gradient. L'obiettivo è modificare il file di input 
# (input_file) in modo da utilizzare al posto dei costi lineari del file originale, il gradiente 
# (vogliamo risolvere il sottoproblema lineare, aka l'approssimazione lineare del problema Taylor 1st ordine).   

def modify_file_with_gradient(input_file, output_file, gradient):
    with open(input_file, 'r') as file:
        lines = file.readlines()

    # inizalizzazione dell'indice per il vettore gradient
    gradient_index = 0

    # modifica delle righe che iniziano con 'a'
    for i in range(len(lines)):
        if lines[i].startswith('a'):
            words = lines[i].split()
            words[-1] = str(gradient[gradient_index])
            gradient_index += 1
            lines[i] = ' '.join(words)

    # creazione nuovo file
    with open(output_file, 'w') as file:
        file.writelines(lines)

In [ ]:
# questo codice serve per calcolare lo step size trovando alpha che minimizzi f (xk + α(dk − xk))
# rispettando il vincolo 0 ≤ α ≤ 1. dove f è la nostra funzione di costo. 

# definizione della funzione quadratica - la nostra f iniziale x^TQx + q
def quadratic_function(alpha, xk, dk, Q, q):

    x_alpha = [x + alpha * (d - x) for x, d in zip(xk, dk)] 
    x_alpha = np.array(x_alpha) 

    # calcola il valore della funzione obiettivo
    return x_alpha.T @ Q @ x_alpha + q @ x_alpha


def find_optimal_alpha(xk, dk, Q, q): # exact line search
   
    objective_function = lambda alpha: quadratic_function(alpha, xk, dk, Q, q) # nostra funzione obiettivo che dipende da alpha 
    result = minimize_scalar(objective_function, bounds=(0, 1), method = 'bounded') # soluzione del problema di minimo con vincolo su alpha che deve stare tra 0 e 1 

    # ritorna il valor ottimo di alpha
    return result.x


In [ ]:

def showModuleFunctionality(mcf):
    vettore_soluzione = {}  
    nmx = mcf.MCFnmax()
    mmx = mcf.MCFmmax()
    pn = mcf.MCFnmax()
    pm = mcf.MCFmmax()

    pU = []
    caps = new_darray(mmx)
    mcf.MCFUCaps(caps)
    for i in range(0, mmx):
        pU.append(darray_get(caps, i))

    pC = []
    costs = new_darray(mmx)
    mcf.MCFCosts(costs)
    for i in range(0, mmx):
        pC.append(darray_get(costs, i))

    pDfct = []
    supply = new_darray(nmx)
    mcf.MCFDfcts(supply)
    for i in range(0, nmx):
        pDfct.append(darray_get(supply, i))

    pSn = []
    pEn = []
    startNodes = new_uiarray(mmx)
    endNodes = new_uiarray(mmx)
    mcf.MCFArcs(startNodes, endNodes)
    for i in range(0, mmx):
        pSn.append(uiarray_get(startNodes, i) + 1)
        pEn.append(uiarray_get(endNodes, i) + 1)

    #print("arc flow")
    length = mcf.MCFm()
    flow = new_darray(length)
    length = mcf.MCFn()
    nms = new_uiarray(length)
    mcf.MCFGetX(flow, nms)

   

    for i in range(0, length):
       # print("flow", darray_get(flow, i), "arc", uiarray_get(nms, i))
        vettore_soluzione[uiarray_get(nms, i)] = darray_get(flow, i)

    return vettore_soluzione  # restituisce il vettore_soluzione alla fine della funzione

In [ ]:
def visualize(k, soluzione_ottima, alpha, prodotto_scalare, k_granularity, tempo, end): 

    # la funzione stampa un summary delle seguenti quantità ad ogni iterazione (oppure ogni 'k_granularity' iterazioni) :
    # - iterazione corrente
    # - step size, 
    # - optimal solution, 
    # - prodotto scalare 

    # alla fine dell'esecuzione verrà mostrato :
    # - prodotto scalare finale,
    # - step size finale
    # - numero totale di iterazioni
    # - tempo di esecuzione totale 
    
    if end:
        print("_"*110)
        print()
        print('Final scalar product: {:>10.2f}  Final step size: {:>10.8f}  Total number iterations: {:>4}'.format(prodotto_scalare, alpha, k))
        print('Total running time: {:>10.2f} seconds'.format(tempo))    
        
    if  k % k_granularity == 0:
        print('Iteration: {:>4}   Step size: {:>10.8f}   Optimal solution: {:>10.6f}   Scalar product grad, d: {:>10.2f}'.format(k, alpha, soluzione_ottima, prodotto_scalare))
    
    return

In [ ]:
def algoritmo(nome_file, epsilon, max_iter, Q, q, u, numero_archi, f_values, step_size_ottimo=True, tau = 0):
    
    k = 0 
    alpha = 1 
    prodotto_scalare = float("inf")
    start_time = time.time()
    tempo_tot = 0
    end = False


    # STEP 0 : inizializzazione di x_0
    seed = 42
    np.random.seed(seed)
    x_old = []

    for u_i in u:
        x_i = random.randint(0, u_i)
        x_old.append(x_i)
    

    while  (abs(prodotto_scalare)>= epsilon and k < max_iter):
        f_x_current = np.dot(np.dot(x_old, Q), x_old) + np.dot(q, x_old)
        f_values.append(f_x_current)

        # STEP 1 : calcolo gradiente
        gradient = (2 * np.dot(Q, x_old)) + q
        gradient = gradient.tolist()
        modify_file_with_gradient(nome_file, 'output.dmx', gradient)

        # risoluzione del problema lineare (ricerca di argmin) con MCF solver
        FILENAME = 'output.dmx'
        f = open(FILENAME,'r')
        inputStr = f.read()
        f.close()
        mcf = MCFSimplex()
        mcf.LoadDMX(inputStr)
        mcf.SolveMCF()
        if mcf.MCFGetStatus() == 0:
            soluzione_ottima = mcf.MCFGetFO()
            visualize(k, soluzione_ottima , alpha, prodotto_scalare, k_granularity = 50, tempo = tempo_tot, end = end)

        else:
            print( "Problem unfeasible!")
         
        vettore_soluzione = showModuleFunctionality(mcf)
     
        sol_x = [0] * numero_archi
        for key in vettore_soluzione:
            if key <= 1000:
                sol_x[key-1] = vettore_soluzione[key]

        # calcolo delle x_bar e determinazione della direzione di ricerca
        x_bar = sol_x
        d = [a - b for a, b in zip(x_bar, x_old)] # direzione

        # STEP 2 : determinazione alpha, step size 

        if step_size_ottimo:
            alpha = find_optimal_alpha(x_old, d, Q, q)
        else:
            alpha = 2 / (2+k)

        # STEP 3 : aggiornamento della posizione
        x_new = []

        for i in range(len(x_old)):
            x_i_new = x_old[i] + alpha * d[i]
            
            if tau == 0: # no trust region stabilization
                x_new.append(x_i_new)
            else:
            
            # TRUST REGION
                # calcolo lower bound lb e upper bound ub della trust region
                lb = x_old[i] - tau
                ub = x_old[i] + tau
                #x_i_new = max(lb, min(x_i_new, min(u[i], ub)))
                x_i_new = max(lb, min(x_i_new, ub)) # senza u 
                x_new.append(x_i_new)

        # check terminazione 
        gradient_per_check = (2 * np.dot(Q, x_new) ) + q
        prodotto_scalare = np.dot(gradient_per_check, d)
    
        # aggiornamento poszione e incremento numero di iterazioni
        x_old = x_new
        k += 1

        if not(abs(prodotto_scalare)>= epsilon and k < max_iter):
            end = True
            end_time = time.time()
            tempo_tot = end_time - start_time
            visualize(k, soluzione_ottima , alpha, prodotto_scalare, k_granularity= 50, tempo = tempo_tot, end = end)
    
    
    return f_values, k

In [ ]:
def pipeline(nome_file_dmx):

    f_values = []
    
    _, numero_archi, u, _, q, _, _ , _ = leggi_file_dimacs(nome_file_dmx)

    valori_f, k_totale = algoritmo(nome_file_dmx, epsilon = 0.1, max_iter = 1000, Q = Q, q = q, u = u, numero_archi = numero_archi, f_values = f_values, step_size_ottimo = True)
    
    return valori_f, k_totale

In [ ]:
def plot_convergenza(f_values, k_tot, Q, nome_file_dmx):
    
    # questa funzione plotta un grafico di convergenza: 
    # viene mostrato l'andamento della convergenza teorica e reale

    # conv teorica  :  f(x)-f(x^*) <=  2LD^2 / (k+3)
    _, _, u, _, _, _, _ , _ = leggi_file_dimacs(nome_file_dmx)
    Q_diag = np.diag(Q)

    # l'autovalore massimo di Q è la costante di convergenza L 
    L = np.max(Q_diag)
    # il diametro dello spazio D è la norma del vettore u 
    D = np.linalg.norm(u) 

    k_values = []  # valori di k (iterazione)
    convergence_values = []  # valori di convergenza teorica

    # simulo i dati teorici (convergenza teorica)
    for k in range(1, k_tot + 1):
        k_values.append(k)
        convergence_values.append((2 * L * D**2) / (k + 3)) # formula converg teorica

    plt.figure(figsize=(10, 6))
    plt.plot(k_values, convergence_values, label="Convergenza Teorica", linestyle='--', linewidth = 3)
    plt.plot(k_values, f_values, label="Convergenza Effettiva", linestyle='-', linewidth = 3)

    plt.xlabel("Iterazione (k)", fontsize = 12)
    plt.ylabel("Valore di Convergenza", fontsize = 12)
    plt.legend()
    plt.title("Convergenza teorica vs effettiva", fontsize = 15)
    plt.grid(True)

    plt.show()

    return